In [1]:
import os
import sys
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Lambda
from keras.engine.topology import Input
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout
from keras.layers import Activation, BatchNormalization, MaxPooling2D
import time
import math
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.engine.topology import Input
from keras import backend as K
K.set_image_dim_ordering('tf')

/Users/ouyangjiahong/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

In [3]:
class GazeNet():
    def __init__(self,
            learning_rate = 0.0001,
            time_step = 32,
            num_classes = 6,
            batch_size = 4,
            data_dim = 4096,):
        self.learning_rate = learning_rate
        self.time_step = time_step
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.data_dim = data_dim
        self.gaussian_sigma = 1
        self.gaussian_weight = self.create_gaussian_weight()
        self.model = self.create_mode()
    
    def create_gaussian_weight(self):
        kernel_size = 15    #same with the shape of the layer before flatten
        kernel_num = 256
        r = (kernel_size - 1) // 2
        sigma_2 = float(self.gaussian_sigma * self.gaussian_sigma)
        pi = 3.1415926
        ratio = 1 / (2*pi*sigma_2)
        
        kernel = np.zeros((kernel_size, kernel_size))
        for i in range(-r, r+1):
            for j in range(-r, r+1):
                tmp = math.exp(-(i*i+j*j)/(2*sigma_2))
                kernel[i+r][j+r] = round(tmp, 3)
        kernel *= ratio
        kernel = np.expand_dims(kernel, axis=2)
        kernel = np.tile(kernel, (1,1,kernel_num))
        print(kernel.shape)
        return kernel
                
    
    def create_mode(self):

        # build the model from the scratch
        input_img = Input((128,128,3,))

        #block 1
        x = Conv2D(96, (5,5), strides=(2, 2), padding='valid')(input_img)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2,2))(x)

#         #block 2
#         x = Conv2D(256, (5,5), padding='same')(x)
#         x = BatchNormalization()(x)
#         x = Activation('relu')(x)
#         x = MaxPooling2D((2,2))(x)

        #block 2
        x = Conv2D(256, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(256, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(256, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2,2))(x)
        print(x)
        
        def multiply_constant(input):
            for i in range(self.batch_size*self.time_step):
                tmp = tf.multiply(tf.cast(input[i], tf.float32), tf.cast(self.gaussian_weight, tf.float32))
                if i == 0:
                    res = tmp
                else:
                    res = tf.stack([res, tmp])
            print(res.shape)
            return res
        
        # fully-connected layers
        x = Lambda(multiply_constant)(x)
        print(x)
        x = Flatten()(x)
        print(x)
        x = Dense(4096, activation = 'relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(4096, activation = 'relu')(x)
        mlp = Dropout(0.5)(x)
        print(mlp)

        def mean_value(input):
            return [np.mean(input[i:i+self.time_step]) for i in range(self.batch_size)]

        # LSTM
#         print mlp
        LSTMLayer = LSTM(128,return_sequences = True)(mlp)
        print(LSTMLayer)
        print("LAMBDA")
        output = Lambda(mean_value)(LSTMLayer)
        model = Model(inputs = input_img,outputs = output)
        adam = optimizers.Adam(lr = self.learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        
        return model
        
#     def train(self):
#         # categorical_labels = to_categorical(int_labels, num_classes=None)
#         pass

#     def load_data(self):
#         print("Hello world")

#     def save_model_weights(self, folder_path, suffix):
#         # Helper function to save your model / weights.
#         self.model.save_weights(folder_path + 'weights-' +  str(suffix) + '.h5')
#         self.model.save(folder_path + 'model-' +  str(suffix) + '.h5')

#     def load_model(self, model_file):
#         # Helper function to load an existing model.
#         self.model = load_model(model_file)

#     def load_model_weights(self,weight_file):
#         # Helper funciton to load model weights.
#         self.model.load_weights(weight_file)

In [4]:
gaze_net = GazeNet()
SVG(model_to_dot(gaze_net.model).create(prog='dot', format='svg'))
plot_model(gaze_net.model, to_file='model.png')

(15, 15, 256)
Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 15, 15, 256), dtype=float32)


ValueError: Shapes must be equal rank, but are 4 and 3
	From merging shape 0 with other shapes. for 'lambda_1/stack_2' (op: 'Pack') with input shapes: [2,15,15,256], [15,15,256].

In [ ]:
print(2)